# Leafletfinder to Edge List

Use Leaflet Finder Code to create an graph based on Oliver's MD trajectory.

**Data:**

In [ ]:
import numpy as np
np.__config__.show() 

The first step in any MDAnalysis script is to load a topology file (which contains a list of particles and possibly additional static data such as bonds or partial charges) and a trajectory file. The trajectory contains the coordinates, which change for each time step. In MDAnalysis, the Universe object ties the topology and the trajectory together and part of
the process of instantiating Universe (topology, trajectory ) is to parse these files.

File Types:
* `.xtc` compressed trajectory file from Gromacs
* `.tpr` topology files

In [13]:
!ls -lh /data/leafletfinder/large/vesicle_1_5M_373*

-rw-r--r-- 1 iparask iparask 71M Nov  4 20:22 /data/leafletfinder/large/vesicle_1_5M_373_first.gro
-rw-r--r-- 1 iparask iparask 71M Nov  4 20:20 /data/leafletfinder/large/vesicle_1_5M_373_last.gro
-rw-r--r-- 1 iparask iparask 90M Nov  4 20:21 /data/leafletfinder/large/vesicle_1_5M_373_stride1000.xtc
-rw-r--r-- 1 iparask iparask 41M Nov  4 20:22 /data/leafletfinder/large/vesicle_1_5M_373.tpr


In [14]:
import MDAnalysis
topology = "/data/leafletfinder/large/vesicle_1_5M_373.tpr"
trajectory = "/data/leafletfinder/large/vesicle_1_5M_373_stride1000.xtc"
u = MDAnalysis.Universe(topology, trajectory)

In [19]:
%time
selection = u.select_atoms("name P*")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [22]:
u

<Universe with 1748952 atoms and 1603206 bonds>

In [29]:
count = 0
for ts in u.trajectory:
    print("Frame: %5d, Time: %8.3f ps" % (ts.frame, u.trajectory.time))
    print("Rgyr: %g A" % (u.atoms.radius_of_gyration(), ))
    count = count + 1 
print "Number of frames: %d"%count 

Frame:     0, Time:    0.000 ps
Rgyr: 652.801 A
Frame:     1, Time: 50000.000 ps
Rgyr: 650.131 A
Frame:     2, Time: 100000.000 ps
Rgyr: 637.096 A
Frame:     3, Time: 150000.000 ps
Rgyr: 627.282 A
Frame:     4, Time: 200000.000 ps
Rgyr: 618.614 A
Frame:     5, Time: 250000.000 ps
Rgyr: 609.713 A
Frame:     6, Time: 300000.000 ps
Rgyr: 599.82 A
Frame:     7, Time: 350000.000 ps
Rgyr: 588.657 A
Frame:     8, Time: 400000.000 ps
Rgyr: 578.532 A
Frame:     9, Time: 450000.000 ps
Rgyr: 564.654 A
Frame:    10, Time: 500000.000 ps
Rgyr: 550.324 A
Frame:    11, Time: 550000.000 ps
Rgyr: 533.978 A
Frame:    12, Time: 600000.000 ps
Rgyr: 516.298 A
Frame:    13, Time: 650000.000 ps
Rgyr: 499.393 A
Frame:    14, Time: 700000.000 ps
Rgyr: 483.763 A
Number of frames: 15


In [23]:
selection

<AtomGroup with 145746 atoms>

In [28]:
selection

In [2]:
import MDAnalysis.analysis.leaflet
L = MDAnalysis.analysis.leaflet.LeafletFinder(u, "name P*", pbc=True, sparse=True)

In [3]:
graph = L.graph

In [24]:
import networkx as NX
NX.write_edgelist(graph,
                  "graph_edges_%d_%d.csv"%(NX.number_of_nodes(graph), NX.number_of_edges(graph)),
                  delimiter=",")

In [26]:
count = 0
for i in NX.connected_components(graph):
    count = count + 1
    
print str(count)

4
